# Unicorn Hybrid Black recorded file streaming

**Save data as .fif**

In [1]:
# import mne
# import numpy as np
# from mne.datasets import eegbci
# import matplotlib.pyplot as plt
# from os import listdir
# from mne.channels import make_standard_montage


# condition = "Offline_Experiment"
# # condition = "Online_Experiment"
# subject_name = "pipo"
# path = "C:\\git\Senior_Thesis\\DataSet\\"+condition+"\\"+ subject_name +"\\notch_EDF\\"
# list_dir = listdir(path)
# raw_each = [0] * len(list_dir)
# for i in range(len(list_dir)):
#     raw_each[i] = mne.io.read_raw_edf(path+list_dir[i],preload = True)
# # raw_edf = mne.concatenate_raws([raw_each[1],raw_each[2]])
    
# raw_edf = mne.concatenate_raws(raw_each)

# eegbci.standardize(raw_edf)  # set channel names
# montage = make_standard_montage("standard_1005")
# raw_edf.set_montage(montage)

# raw_edf.save("Unicorn_MI.fif", overwrite=True)

**Using LSL to stream**

In [2]:
import mne
import numpy as np
from mne.channels import make_standard_montage
import matplotlib.pyplot as plt
from mne.datasets import eegbci

raw_eeg = mne.io.read_raw_fif("Unicorn_MI.fif")

Opening raw data file Unicorn_MI.fif...
    Range : 0 ... 311249 =      0.000 ...  1244.996 secs
Ready.


C:\Users\pipo_\AppData\Local\Temp\ipykernel_31344\1704288971.py:7: RuntimeWarning: This filename (Unicorn_MI.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_eeg = mne.io.read_raw_fif("Unicorn_MI.fif")


In [3]:
raw_eeg

Measurement date,"June 24, 2024 21:33:52 GMT"
Experimenter,Unknown
Participant,X
Digitized points,11 points
Good channels,8 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [4]:
events_raw, _ = mne.events_from_annotations(raw_eeg)

print(events_raw)

Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up']
[[  7500      0      1]
 [  8250      0      2]
 [  8500      0      2]
 ...
 [308500      0      2]
 [308750      0      2]
 [309000      0      2]]


# Push Chunk

In [5]:
from mne_realtime import LSLClient, MockLSLStream
from pylsl import StreamInfo, StreamOutlet
import time
import mne

# Assuming raw_eeg is already created
# Create the LSL stream for the raw EEG data
eeg_info = StreamInfo(name='EEGStream', type='EEG', channel_count=raw_eeg.info['nchan'],
                      nominal_srate=raw_eeg.info['sfreq'], channel_format='float32')
eeg_outlet = StreamOutlet(eeg_info)

# Host name and wait time
host = 'My_mne_stream'
wait_max = 5

def send_eeg_chunks(raw_eeg, chunk_size):
    """ Send raw EEG data in chunks through LSL. """
    sfreq = raw_eeg.info['sfreq']  # Sampling frequency
    n_samples = len(raw_eeg.times)  # Total number of samples in the raw data
    data, _ = raw_eeg[:, :]  # Get all the EEG data as an array

    for start_idx in range(0, n_samples, chunk_size):
        # Determine the end of the chunk
        end_idx = min(start_idx + chunk_size, n_samples)
        
        # Extract the chunk of data
        chunk = data[:, start_idx:end_idx]

        print(chunk.shape)

        # Push the chunk of raw EEG data to the LSL stream
        eeg_outlet.push_chunk(chunk.T.tolist())  # Transpose for proper LSL format
        print(f"Sent chunk of {end_idx - start_idx} samples")

        # Simulate real-time delay for the chunk
        chunk_duration = (end_idx - start_idx) / sfreq
        time.sleep(chunk_duration)

if __name__ == '__main__':
    while True:
        # Set the chunk size (e.g., 100 samples per chunk)
        chunk_size = 250
        # Send raw EEG data in chunks
        send_eeg_chunks(raw_eeg, chunk_size)

(8, 250)
Sent chunk of 250 samples


C:\Users\pipo_\AppData\Local\Temp\ipykernel_31344\3363593252.py:1: FutureWarning: MNE-realtime is discontinued in favor of the Lab Streaming Layer real-time interface availabe in MNE-lsl: https://mne.tools/mne-lsl/
  from mne_realtime import LSLClient, MockLSLStream


(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk of 250 samples
(8, 250)
Sent chunk 

KeyboardInterrupt: 